how this notebook works

In [ ]:
# dependencies

import pandas as pd

from ModelBuilderMethods import scaleColumns, encodeColumns

setting up a dataset

In [1]:
%pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 3.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.8 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


splitting the dataset

balancing the dataset https://imbalanced-learn.org/stable/



### normalization / scaling
some blurb about scalers  
0 MinMaxScaler           https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html  
1 MaxAbsScaler           https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html  
2 StandardScaler         https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html  
3 RobustScaler           https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html  
4 Normalizer             https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html  
5 PowerTransformer       https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html  
6 QuantileTransformer    https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html  

In [ ]:
df = pd.DataFrame()
scaled = scaleColumns(df, ['max_temp'], None, 1)

categorical values one-hot encoding  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html  


selecting a model

print model performance metrics on test data